In [1]:
import jinja2
import os

env=jinja2.Environment(loader=jinja2.FileSystemLoader(os.path.join(os.getcwd(), 'templates')))

In [2]:
class TooManyMatchesError(Exception):
    pass

class SymbolNotFoundError(Exception):
    pass

class SymbolStore:
    def __init__(self):
        self.symbols = list()
    
    def _get_by(self, **kwargs):
        for s in self.symbols:
            for k, v in kwargs.items():
                if s.get(k, None) != v:
                    break
            else:
                yield s


    def get_first(self, **kwargs):
        try:
            return next(self._get_by(**kwargs))
        except StopIteration as exc:
            raise SymbolNotFoundError("Symbol not found") from exc
    
    def get_one(self, **kwargs):
        i = self._get_by(**kwargs)
        
        try:
            s = next(i)
        except StopIteration as exc:
            raise SymbolNotFoundError("Symbol not found") from exc
        
        try:
            next(i)
        except StopIteration:
            return s
        else:
            raise TooManyMatchesError("More than one match while calling `get_one`")
    
    def get_all(self, **kwargs):
        return list(self._get_by(**kwargs))
    
    def get_or_create(self, **kwargs):
        try:
            s = self.get_one(**kwargs)
        except SymbolNotFoundError:
            s = kwargs.copy()
            self.symbols.append(s)
        return s

In [3]:
import regex
from itertools import groupby
from uuid import uuid4

class Token(str):    
    def __repr__(self):
        return f"{self.__class__.__name__}({super().__repr__()})"
        
class Word(Token):
    pass

class Acronym(Word):
    pass

class Separator(Token):
    pass

class Name:
    _camel_case_regex = regex.compile(r"^([a-z]+)((:?[A-Z0-9][A-Za-z0-9]*?)*)$")
    _full_camel_case_regex = regex.compile(r"^([A-Z][a-z0-9A-Z]*?)((:?[A-Z0-9][A-Za-z0-9]*?)*)$")
    
    def __init__(self, *tokens, **meta):
        self._tokens = list(tokens)
        self._meta = meta

    @classmethod
    def from_snake_case_all_caps(cls, text):
        tokens = []
        for t in text.split('_'):
            tokens.append(Word(t))
            tokens.append(Separator('_'))
            
        return cls(*tokens[:-1], original=text)

    @staticmethod
    def _scan_camel_case_regex(regex, text):
        tokens = []
        if (match := regex.match(text)) is not None:
            captures = [match.group(1)] + list(match.captures(3))
            for _, ws in groupby(captures, key=lambda k: str(k.isalpha()) if len(k) == 1 else repr(uuid4())):
                group = list(ws)
                if len(group) > 1:
                    tokens.append(Acronym(''.join(group)))
                else:
                    tokens.append(Word(group[0]))
            return tokens
        else:
            raise ValueError("Not in camel case format")

    @classmethod
    def from_camel_case(cls, text):
        return cls(*cls._scan_camel_case_regex(cls._camel_case_regex, text), original=text)
    
    @classmethod
    def from_full_camel_case(cls, text):
        return cls(*cls._scan_camel_case_regex(cls._full_camel_case_regex, text), original=text)

    def to_snake_case(self):
        words = []
        for t in self._tokens:
            if isinstance(t, Word):
                words.append(t.lower())
        return '_'.join(words)
    
    def to_snake_case_all_caps(self):
        return self.to_snake_case().upper()
    
    def to_camel_case(self):
        words = []
        is_first_word = True
        for t in self._tokens:
            if isinstance(t, Word):
                if is_first_word:
                    words.append(t.lower())
                    is_first_word = False
                else:
                    words.append(t.capitalize())
        return ''.join(words)        
    
    def to_full_camel_case(self):
        return ''.join(t.capitalize() for t in self._tokens if isinstance(t, Word))

    def __repr__(self):
        return f"Name(*{repr(self._tokens)}, **{repr(self._meta)})"

In [10]:
class CodeRenderer:
    def __init__(self, js_template="empty.tmpl.js", ps_template="empty.tmpl.ps", **extra):
        self.js_template = env.get_template(js_template)
        self.ps_template = env.get_template(ps_template)
        self.extra = extra
        
    def render_js_part(self, **kwargs):
        return self.js_template.render(this=self, **self.extra, **kwargs)

    def render_ps_part(self, **kwargs):
        return self.ps_template.render(this=self, **self.extra, **kwargs)


class Module(CodeRenderer):
    """A PS module"""
    def __init__(self, path, declarations=None, **kwargs):
        self.path = path
        if declarations is None:
            declarations = list()
        self.declarations = declarations  # [Definition]
        super().__init__(**kwargs)
    
    @property
    def exported(self):
        return [d.export_code for d in self.declarations if d.exported]


class Definition(CodeRenderer):
    def __init__(self, name, exported=False, **kwargs):
        self.name = name
        self.exported = exported
        super().__init__(**kwargs)
    
    @property
    def export_code(self):
        return self.name

    def __repr__(self):
        return f"{self.__class__.__name__}({self.name})"


class UnionType(Definition):
    def __init__(self, constructors=None, **kwargs):
        if constructors is None:
            constructors = list()
        self.constructors = constructors  # [name]
        super().__init__(**kwargs)
    
    @property
    def export_code(self):
        return f"{self.name}(..)"


class Function(Definition):
    """A PS function"""
    def __init__(self, **kwargs):
        self.parameters = dict()  # name: type_resolver
        self.result_type = None  # type_resolver
        self.pointfree_parameter_types = list()  # [type_resolver]
        super().__init__(**kwargs)

    def render_js_part(self, **kwargs):
        return super().render_js_part(function=self, **kwargs)


class ForeignFunctionImport(Definition):
    def __init__(self, **kwargs):
        self.parameter_types = list()
        self.result_type = None
        super().__init__(**kwargs)
    
    def render_js_part(self, **kwargs):
        return super().render_js_part(function=self, **kwargs)


class ForeignTypeImport(Definition):
    pass

In [11]:
import json

official_api = SymbolStore()

data = json.load(open('gasapi.json'))

for d in data:
    s = official_api.get_or_create(type=d['type'], url=d['url'], name=d['name'])
    s['methods'] = d['methods']
    s['properties'] = d['properties']

In [27]:
purescript_api = SymbolStore()

for enum in official_api.get_all(type='enum'):
    module = Module(
        path=f"Data.Google.AppsScript.{enum['name']}",
        js_template="module.tmpl.js",
        ps_template="module.tmpl.ps",
        enum=enum)
    
    enum_parent = [c for c in official_api.get_all(type='class')
                   if any(p['url'] == enum['url']
                          for p in c['properties'])]

    # assert len(enum_parent) < 2, f"{enum['name']} has {len(enum_parent)} parents: {[p['name'] for p in enum_parent]}"
    if (num_parents := len(enum_parent)) != 1:
        print(num_parents, enum['url'])

    # The union declaration
    union_name = enum['name']  # Already in full camel case
    constructors = [Name.from_snake_case_all_caps(p['name']).to_full_camel_case()
                    for p in enum['properties']]
    union = UnionType(
        name=union_name,
        exported=True,
        constructors=constructors,
        ps_template="enum_union.tmpl.ps"
    )
    module.declarations.append(union)

    # The foreign type import
    foreign_type_name = f"Foreign{union_name}"
    foreign_type = ForeignTypeImport(
        name=foreign_type_name,
        exported=True,
        ps_template="enum_foreign_type.tmpl.ps"
    )
    module.declarations.append(foreign_type)
    
    foreign_constructors = list()
    for p in enum['properties']:
        foreign_fn_name = f"foreign{enum['name']}_{Name.from_snake_case_all_caps(p['name']).to_full_camel_case()}"
        foreign_fn = ForeignFunctionImport(
            name=foreign_fn_name,
            js_template="enum_property.tmpl.js",
            ps_template="enum_property.tmpl.ps",
            property=p,
            enum=enum,
            parents=enum_parent,
            foreign_type=foreign_type,
        )
        foreign_fn.result_type = lambda: foreign_type
        module.declarations.append(foreign_fn)
        foreign_constructors.append(foreign_fn)
    
    #
    # JS2PS & PS2JS
    #
    transform_fn_prefix_name = Name.from_full_camel_case(enum['name']).to_camel_case()
    
    # A foreign function to transform from a JS value to PS
    foreign_fn_js2ps_name = f"{transform_fn_prefix_name}FromForeignImpl"
    foreign_fn_js2ps = ForeignFunctionImport(
        name=foreign_fn_js2ps_name,
        js_template="enum_js2ps.tmpl.js",
        ps_template="enum_js2ps.tmpl.ps",
        union=union,
        enum=enum,
        parents=enum_parent,
        foreign_type=foreign_type,
    )
    foreign_fn_js2ps.result_type = lambda: union
    module.declarations.append(foreign_fn_js2ps)
    
    # A wrapper for the last function
    fn_js2ps_name = f"{transform_fn_prefix_name}FromForeign"
    fn_js2ps = Function(
        name=fn_js2ps_name,
        exported=True,
        ps_template="enum_js2ps_wrapper.tmpl.ps",
        foreign_type=foreign_type,
        union=union,
        foreign_fn_js2ps=foreign_fn_js2ps
    )
    fn_js2ps.pointfree_parameter_types.append(lambda: foreign_type)
    fn_js2ps.result_type = lambda: union
    module.declarations.append(fn_js2ps)

    # A function to transform from a PS value to JS
    fn_ps2js_name = f"{transform_fn_prefix_name}ToForeign"
    fn_ps2js = Function(
        name=fn_ps2js_name,
        exported=True,
        ps_template="enum_ps2js.tmpl.ps",
        foreign_type=foreign_type,
        foreign_constructors=foreign_constructors,
        union=union
    )
    fn_ps2js.pointfree_parameter_types.append(lambda: union)
    fn_ps2js.result_type = lambda: foreign_type
    module.declarations.append(fn_ps2js)
    
    # Save module
    purescript_api.get_or_create(source=enum['url'], type='module')['definition'] = module

# module=purescript_api.get_first(source='https://developers.google.com/apps-script/reference/base/mime-type')['definition']
module=purescript_api.get_all()[10]['definition']
print(f"{'='*40} PS {'='*40}")
print(module.render_ps_part(api=purescript_api, module=module))
print(f"{'='*40} JS {'='*40}")
print(module.render_js_part(api=purescript_api, module=module))


0 https://developers.google.com/apps-script/reference/charts/picker-values-layout
0 https://developers.google.com/apps-script/reference/charts/orientation
0 https://developers.google.com/apps-script/reference/charts/match-type
0 https://developers.google.com/apps-script/reference/card-service/switch-control-type
0 https://developers.google.com/apps-script/reference/card-service/display-style
2 https://developers.google.com/apps-script/reference/base/weekday
2 https://developers.google.com/apps-script/reference/base/color-type
2 https://developers.google.com/apps-script/reference/base/month
2 https://developers.google.com/apps-script/reference/base/button-set
0 https://developers.google.com/apps-script/reference/base/mime-type
======================================== PS ========================================
{--
This module has been generated automatically by gas-doc-scrapper
with data obtained from:

https://developers.google.com/apps-script/reference/script/trigger-source

         